In [1]:
from mido import MidiFile
import warnings
warnings.filterwarnings("ignore")

In [2]:
from commons import *

In [3]:
## Importa la tabla que parsea numeros de notas a nombre y octava
import pandas as pd
import numpy as np
import os, sys
from contextlib import suppress
import pickle  
with open('dict_columns_total.pkl', 'rb') as df_pkl:
        mask_columns = pickle.load(df_pkl)

In [75]:
files_path = str(sys.path[0]) + '\Full_MIDI'

dict = {}
count_files = 0
      
for root, dirs, files in os.walk(files_path, topdown=False):
    for name_file in files:
        dict[count_files] = {"file": os.path.join(root, name_file), "file_name": name_file}
        count_files = count_files + 1

df_files = pd.DataFrame.from_dict(dict, "index")
df_files.sort_values('file_name',inplace=True)
df_files.reset_index(inplace=True,drop=True)
display(df_files)

,file,file_name
0,C:\dh\FULL_MIDI\Full_MIDI\Categories\unclassif...,'74-'75.mid
1,C:\dh\FULL_MIDI\Full_MIDI\Categories\unclassif...,'74-'75.mid
2,C:\dh\FULL_MIDI\Full_MIDI\Categories\unclassif...,'74-'75.txt
3,C:\dh\FULL_MIDI\Full_MIDI\Categories\unclassif...,'74-'75.txt
4,C:\dh\FULL_MIDI\Full_MIDI\Categories\unclassif...,'74-'75_2.mid
...,...,...
166912,C:\dh\FULL_MIDI\Full_MIDI\Z\Z\zz_top-tush.mid,zz_top-tush.mid
166913,C:\dh\FULL_MIDI\Full_MIDI\Z\Z\zzdq.MID,zzdq.MID
166914,C:\dh\FULL_MIDI\Full_MIDI\Z\Z\zzgsv.mid,zzgsv.mid
166915,C:\dh\FULL_MIDI\Full_MIDI\Z\Z\zzgundam.mid,zzgundam.mid


In [78]:
files_path = str(sys.path[0]) + '\Full_MIDI\\'
FILE = "FULL_MIDI_por_proecesar.csv"
df_files = pd.read_csv(FILE,sep=",")
df_files.sort_values('file_name',inplace=True)
df_files.reset_index(inplace=True,drop=True)
df_files['file'] = df_files['full_path'].apply(lambda x: files_path +  x.replace('/','\\'))
del df_files['path']
del df_files['full_path']
df_files

,file,file_name
0,C:\dh\FULL_MIDI\Full_MIDI\Classical Archives -...,'Lieder ohne Worte' Book 1 op1 n1.mid
1,C:\dh\FULL_MIDI\Full_MIDI\Classical Archives -...,'Lieder ohne Worte' Book 2 op30 n3.mid
2,C:\dh\FULL_MIDI\Full_MIDI\Classical Archives -...,"(2542)Prelude opus.28, No.16 in B flat minor.mid"
3,C:\dh\FULL_MIDI\Full_MIDI\Classical Archives -...,-Study No.1 opus.105.mid
4,C:\dh\FULL_MIDI\Full_MIDI\Classical Archives -...,-Study No.2 opus.105.mid
...,...,...
76980,C:\dh\FULL_MIDI\Full_MIDI\Z\Z\zorro2.mid,zorro2.mid
76981,C:\dh\FULL_MIDI\Full_MIDI\Guitar_midkar.com_MI...,zuccherina_polka-it_ab.mid
76982,C:\dh\FULL_MIDI\Full_MIDI\Z\Z\zucchero-ali_d_o...,zucchero-ali_d_oro.mid
76983,C:\dh\FULL_MIDI\Full_MIDI\Z\Z\zucchero-music_i...,zucchero-music_in_me.mid


In [96]:
BATCH_SIZE = 4000
files_from = 55258
files_to = 76985
CSV_SAVE = 'midi_stats_vX7'
print(files_from,files_to)

55258 76985


In [92]:
dict = {}
count_files = 0
count_batch = 0
errors_arr = []

In [97]:
for index, row in df_files[files_from:files_to].iterrows():
    with suppress(Exception):
        if ((count_files % BATCH_SIZE == 0) and (count_files > 0)):
            print('SAVING!!')
            df_midi_stats = pd.DataFrame.from_dict(dict, "index")   
            keep_columns = (df_midi_stats.isnull().sum() / df_midi_stats.shape[0]) <= 0.98
            #           keep_columns = list(set(mask_columns) & set(df_midi_stats.columns))
            df_midi_stats.loc[:,keep_columns].to_csv(CSV_SAVE + '.csv')
            print('SAVED!!')
            del df_midi_stats
            count_batch = count_batch + 1

        dict[count_files] = get_theme_stats(row.file, row.file_name)[0] 

        count_files = count_files + 1
        print('success', str(count_files), '/', index + 1)
        print('batch', str(count_batch))

zuccherina_polka-it_ab.mid G
sec: 139.49999999999923
comp: 139
success 20684 / 76982
batch 5


In [98]:
# keep_columns = list(set(mask_columns) & set(df_midi_stats.columns))
df_midi_stats = pd.DataFrame.from_dict(dict, "index")   
keep_columns = (df_midi_stats.isnull().sum() / df_midi_stats.shape[0]) <= 0.98
df_midi_stats.loc[:,keep_columns].to_csv(CSV_SAVE + '.csv')
print('SAVED!!')
del df_midi_stats

SAVED!!


In [82]:
count_files

12589

In [85]:
df_files[df_files.file_name == 'Moreno-Torroba_Castles_Spain_Simancas.mid']

,file,file_name
23999,C:\dh\FULL_MIDI\Full_MIDI\Classical_Guitar_cla...,Moreno-Torroba_Castles_Spain_Simancas.mid
